In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch import nn
import torch.nn.functional as F

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


Prepare Train / test data sets

In [2]:
num_workers = 0
batch_size = 32
valid_size = 0.2

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=train_transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=test_transform)

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

  0%|          | 0/170498071 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Simple CNN with two convolutions

7 (weighted) layer 이상을 쌓으려면... pool 한번에 절반씩 줄어들기 때문에 pool을 계속 하는 것도 이상함. pool 네 번이면 2x2로 작아짐. 

pooling 없이 conv만 하면? -- pooling이나 conv나 계산량이 비슷하므로 conv를 자꾸 하는 것도 나쁘지 않을 듯. 

In [19]:
from ResNet20 import ResNet, BasicBlock

In [20]:
model = ResNet(BasicBlock,[1,1,1])

model.to('cuda')
train_on_gpu=True

print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (shortcut): Sequential()
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, e

## Optimizer

In [21]:
import torch.optim as optim
import matplotlib.pyplot as plt 

FastAI training

In [24]:
from fastai.vision.all import *
import torchvision

import fastai
from fastai.optimizer import OptimWrapper

from fastai.data.core import DataLoaders
from fastai.learner import Learner
from fastai.callback.progress import ProgressCallback

dls = DataLoaders(train_loader, test_loader)

criterion = nn.CrossEntropyLoss()
learn = Learner(dls, model, loss_func=criterion)#, opt_func=opt_func)#, cbs=[CudaCallback])

In [25]:
learn.fine_tune(25)

epoch,train_loss,valid_loss,time
0,1.364833,1.323575,00:14


epoch,train_loss,valid_loss,time
0,1.106656,1.057492,00:14
1,1.041451,1.009262,00:14
2,1.034626,0.995960,00:14
3,1.003910,1.028591,00:14
4,0.930804,1.010954,00:14
5,0.894462,0.893596,00:14
6,0.909983,0.915636,00:14
7,0.850525,0.890391,00:14
8,0.833577,0.786139,00:14
9,0.777463,0.896481,00:14


In [16]:
torch.save(model.state_dict(), "models/ResNet8.pt")

## Validation accuracy

In [26]:
# track test loss
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, 
    num_workers=num_workers)

test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    output = model(data)
    loss = criterion(output, target)
    test_loss += loss.item()*data.size(0)
    _, pred = torch.max(output, 1)    
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())

    for i in range(len(data)):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.565315

Test Accuracy of airplane: 82% (820/1000)
Test Accuracy of automobile: 90% (902/1000)
Test Accuracy of  bird: 70% (702/1000)
Test Accuracy of   cat: 63% (636/1000)
Test Accuracy of  deer: 78% (784/1000)
Test Accuracy of   dog: 75% (751/1000)
Test Accuracy of  frog: 85% (856/1000)
Test Accuracy of horse: 82% (828/1000)
Test Accuracy of  ship: 92% (921/1000)
Test Accuracy of truck: 88% (884/1000)

Test Accuracy (Overall): 80% (8084/10000)


## 성능 

(Conv - BN - Avg Pool)
conv 5x5, same padding = 67
conv 3x3, same padding = 68
conv 3x3, valid padding = 67



--------------------------

relu + maxpool: ~62%

relu + avgpool: ~58% -- OK, maxpool -> avgpool은 큰 문제 없음. 

approx. relu + avgpool: 52% !! 

approx. relu + avgpool + BN (2Conv + 3FC, 20 epoch): 58% 정도? 

approx. relu + avgpool + BN (2Conv + 2FC, 50 epoch: 59% 


## Number of trainable parameters

In [9]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(f"We have to train {params} parameters")

We have to train 62006 parameters
